# 畳み込みニューラルネットワーク

## 畳み込み層

- 畳み込み層: CNN（Convolutional Neural Network）で新たに取り入れられる層の一つ。
- 特徴マップ(feature map): 畳み込み層における入出力データのこと
- 入力特徴マップ(input feature map): 畳み込み層における入力データのこと
- 出力特徴マップ(output feature map): 畳み込み層における出力データのこと

http://www.hpc.co.jp/AboutDeepLearning.html
![http://www.hpc.co.jp/images/DL_kaisetsu_09.png](http://www.hpc.co.jp/images/DL_kaisetsu_09.png)

http://pythonskywalker.hatenablog.com/entry/2016/12/26/164545  
![https://cdn-ak.f.st-hatena.com/images/fotolife/r/riikku/20161226/20161226141944.png](https://cdn-ak.f.st-hatena.com/images/fotolife/r/riikku/20161226/20161226141944.png)

### 全結合層の問題点

これまでのニューラルネットワークは隣接する層のすべてのニューロンとニューロンの間で結合がある（fully-connected; 全結合;）状態だった。  
我々はこれまでそれぞれの結合部をAffineレイヤ（行列の内積計算）で実装しReLUにて発火させた。これは何の考慮もせずに入力値をそのまま行列に変換してニューラルネットワークを構築したことになる。つまり今回のMNISTの手書き文字の場合、元々あった縦・横などの「データの形状情報」やRGB（ピクセルの色合いの順序情報;R要素・G要素・B要素の区別;）が失われてしまうことになる。  
__データをそのまま全結合層の入力（行列）として扱った場合3次元以上の情報は失われてしまう！__  
例えば、画像を行列として扱った場合に失われる情報としては以下が考えられる
- ピクセルのR,G,B,Aのそれぞれの濃度
- 縦・横のピクセル幅
- 上下左右で連続したピクセル間の値の推移

### 畳み込み演算

畳み込み演算は画像処理で言う「フィルター演算」にあたる。  
http://www.hpc.co.jp/AboutDeepLearning.html  
![http://www.hpc.co.jp/images/DL_kaisetsu_10.png](http://www.hpc.co.jp/images/DL_kaisetsu_10.png)

「フィルター」はカーネルとも呼ばれ、入力データに対してあるウインドウサイズの区画で積和演算を行う。全結合のニューラルネットワークでは行列が重みパラメータとなっていたが、CNNの場合はこのフィルターの乗算内容が「重み」に対応する。  
積和演算の結果は各ウィンドウの位置ごとに取得され、結果は新たな出力データとなる。  
その出力データの行列にスカラをスカラ加算（行列の要素全てに等しく加算）することでバイアスも実現する。  
バイアスはスカラなので、1 x 1の行列となる。  

### パディング

入力データの大外にダミーの枠を用意することで、積和演算結果のサイズを調整できる。  
畳み込み演算を繰り返すと段々出力サイズが小さくなるので、最終的な出力サイズが1になることを防ぐことができる。  
畳み込み演算回数（隠れ層の数）によっては不要の場合がある。

### ストライド

フィルターの適用する位置の間隔のこと。積和演算が一回終わった時点でウィンドウがスライドする幅。  
パディングとは逆に、ストライドは大きくすると積和演算結果の出力サイズは小さくなる。

#### ストライドと出力サイズの関係

- 出力サイズ ($OH = \frac{H+2P-FH}{S} + 1, OW = \frac{W+2P-FW}{S} + 1$) 
    - 入力サイズ $(H, W)$
    - フィルターサイズ $(FH, FW)$
    - パディング $P$
    - ストライド $S$
  
ただし、ストライドで割り切れない値の場合は長方形の形にならない（行列において欠損データが発生する）為
割り切れない値を丸めるかダミー値を挿入すること。

### 3次元データの畳み込み演算

通常の畳込み演算にチャンネル（奥行き）を加えたもの。  
- 適用フィルターの枚数（=重み） x 入力データ = 出力チャンネル数となる。
- バイアスは（フィルター数, 1, 1）という形でブロードキャスト演算により適用される。

http://pythonskywalker.hatenablog.com/entry/2016/12/26/164545  
![https://camo.qiitausercontent.com/24262ae92d579768024ae0a7a1d00271d66e8917/68747470733a2f2f71696974612d696d6167652d73746f72652e73332e616d617a6f6e6177732e636f6d2f302f32353939302f66333461363337322d393730632d636365382d353861642d3139316432373566656635372e706e67](https://camo.qiitausercontent.com/24262ae92d579768024ae0a7a1d00271d66e8917/68747470733a2f2f71696974612d696d6167652d73746f72652e73332e616d617a6f6e6177732e636f6d2f302f32353939302f66333461363337322d393730632d636365382d353861642d3139316432373566656635372e706e67)

### 畳み込み演算のバッチ演算

単にN個のデータを重ねただけ。（4次元データの演算）

http://pythonskywalker.hatenablog.com/entry/2016/12/26/164545  
![https://cdn-ak.f.st-hatena.com/images/fotolife/r/riikku/20161226/20161226164341.png](https://cdn-ak.f.st-hatena.com/images/fotolife/r/riikku/20161226/20161226164341.png)

## プーリング層

プーリング層(Pooling)は引数フィルターなどを持たずに、関数のみで入力を縦・横の空間を狭くして出力する演算のこと。  
一般的にはウィンドウサイズ＝ストライドとなるように設定する。  
MAXプーリングは以下のような形で演算がなされる。他にも、Averageプーリングなどがある。

https://indoml.com/2018/03/07/student-notes-convolutional-neural-networks-cnn-introduction/  
![https://indoml.files.wordpress.com/2018/03/pooling-layer3.png?w=648&h=180)](https://indoml.files.wordpress.com/2018/03/pooling-layer3.png?w=648&h=180)

### プーリング層の特徴

#### 学習するパラメータが無い

プーリング層は畳み込みそうと違い学習パラメータを持たない。（層自体が成長しない）

##### チャンネル層は変化しない

プーリング層を通過しても入力データと出力データのチャンネル数は変化しない。

##### 微小な位置変化に対してロバスト（頑健）

入力データの小さなズレに対してもプーリング層は同じような結果を返す。  
入力データが多少ズレていてもウィンドウサイズに収まっている程度のズレであれば吸収できる。

## Convolution／Poolingレイヤの実装

### 4次元配列

CNNは高さx横幅xチャンネルxデータ数の四次元セットと言える。

### im2colによる展開

畳み込み演算はim2colを使用して入力データを単純な行列の計算に落とし込む方法を使うと線形代数ライブラリで高速処理できるようになる。

http://narusawa-a-hiyoko.hatenablog.com/entry/2016/04/12/042039
![https://cdn-ak.f.st-hatena.com/images/fotolife/r/riikku/20161226/20161226180402.png](https://cdn-ak.f.st-hatena.com/images/fotolife/r/riikku/20161226/20161226180402.png)

### Convolutionレイヤの実装

#### im2colの実装

In [14]:
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """

    Parameters
    ----------
    input_data : (データ数, チャンネル, 高さ, 幅)の4次元配列からなる入力データ
    filter_h : フィルターの高さ
    filter_w : フィルターの幅
    stride : ストライド
    pad : パディング

    Returns
    -------
    col : 2次元配列
    """
    N, C, H, W = input_data.shape #データ数、チャンネル(奥行き)、高さ、幅
    out_h = (H + 2*pad - filter_h)//stride + 1#出力高さを計算
    out_w = (W + 2*pad - filter_w)//stride + 1#出力幅を計算

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')#0埋めでパディング
    #no.pad(array, pad_width, mode=None, **kwargs)
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))#出力用0埋め二次元データのハコを作成
    #np.zeros(shape, dtype=float, order='C')

    for y in range(filter_h):#ストライドを反映させる
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    #XXX: ここが分からん！！！
    return col

In [17]:
import sys, os
import numpy as np
sys.path.append(os.pardir)
from common.util import im2col

x1 = np.random.rand(1,3,7,7)
print(x1.shape)
#    1: データ数(バッチ数)
#    3: チャンネル(奥行き)
#    7: 高さ
#    7: 幅
#4次元配列からなる入力データ

col1 = im2col(x1, 5, 5, stride=1, pad=0)
#    x1 : (データ数(バッチ数), チャンネル(奥行き), 高さ, 幅)の4次元配列からなる入力データ
#    5 : フィルターの高さ
#    5 : フィルターの幅
#    1 : ストライド
#    0 : パディング
print(col1.shape)
# 9 = #XXX: ここが分からん！
# 75 = (チャンネル(奥行き) x フィルターの高さ x フィルターの高さ)

x2 = np.random.rand(10,3,7,7)
print(x2.shape)

col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)
# 90 = #XXX: ここが分からん！
# 75 = (チャンネル(奥行き) x フィルターの高さ x フィルターの高さ)


(1, 3, 7, 7)
(9, 75)
(10, 3, 7, 7)
(90, 75)


__畳み込み層をConvolutionという名前のクラスで実装する__

In [18]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
        # 中間データ（backward時に使用）
        self.x = None   
        self.col = None
        self.col_W = None
        
        # 重み・バイアスパラメータの勾配
        self.dW = None
        self.db = None

    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = 1 + int((H + 2*self.pad - FH) / self.stride)
        out_w = 1 + int((W + 2*self.pad - FW) / self.stride)

        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T

        out = np.dot(col, col_W) + self.b
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)

        self.x = x
        self.col = col
        self.col_W = col_W

        return out

    def backward(self, dout):
        FN, C, FH, FW = self.W.shape
        dout = dout.transpose(0,2,3,1).reshape(-1, FN)

        self.db = np.sum(dout, axis=0)
        self.dW = np.dot(self.col.T, dout)
        self.dW = self.dW.transpose(1, 0).reshape(FN, C, FH, FW)

        dcol = np.dot(dout, self.col_W.T)
        dx = col2im(dcol, self.x.shape, FH, FW, self.stride, self.pad)

        return dx